In [19]:
import requests
import pandas as pd
import numpy as np
import folium
from folium.plugins import MiniMap

In [20]:
def conv_location(region, page_num):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    params = {'query' : region, 'page' : page_num}
    headers = {'Authorization' : 'KakaoAK @@@@@'}
    
    places = requests.get(url, params = params, headers = headers).json()['documents']
    total = requests.get(url, params = params, headers = headers).json()['meta']['total_count']
    
    if total > 45:
        print(total, '개 중 45개 데이터밖에 가져오지 못했습니다!')
    else:
        print('모든 데이터를 가져왔습니다!')
    return places

In [21]:
def conv_info(places):
    X = []
    Y = []
    stores = []
    road_address = []
    place_url = []
    ID = []
    for place in places:
        X.append(float(place['x']))
        Y.append(float(place['y']))
        stores.append(place['place_name'])
        road_address.append(place['road_address_name'])
        place_url.append(place['place_url'])
        ID.append(place['id'])
    
    ar = np.array([ID, stores, X, Y, road_address, place_url]).T
    df = pd.DataFrame(ar, columns = ['ID', 'stores', 'X', 'Y', 'road_address', 'place_url'])
    
    return df

In [22]:
def keywords(location):
    df = None
    for local in location:
        for page in range(1, 4):
            local_name = conv_location(local, page)
            local_conv_info = conv_info(local_name)
            
            if df is None:
                df = local_conv_info
            elif local_conv_info is None:
                continue
            else:
                df = pd.concat([df, local_conv_info], join = 'outer', ignore_index = True)
        
    return df

In [23]:
def make_map(dfs):
    m = folium.Map(location = [37.5642, 127.0016],
                  zoom_start = 12)
    minimap = MiniMap()
    m.add_child(minimap)
    
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i], df['X'][i]],
                     tooltip = dfs['stores'][i],
                     ).add_to(m)
    return m

In [29]:
location = ['서울시 편의점']
df = keywords(location)
df = df.drop_duplicates(['ID'])
df = df.reset_index()

make_map(df)
# df

9417 개 중 45개 데이터밖에 가져오지 못했습니다!
9417 개 중 45개 데이터밖에 가져오지 못했습니다!
9417 개 중 45개 데이터밖에 가져오지 못했습니다!


In [27]:
import requests 
url = 'https://dapi.kakao.com/v2/local/search/keyword.json' 
params = {'query': "서울시 편의점",'page': 1} 
headers = {"Authorization": "KakaoAK @@@@@@"}
places = requests.get(url, params=params, headers=headers).json()['documents']
total = requests.get(url, params=params, headers=headers).json()['meta']['total_count'] 
if total > 45: 
    print(total,'개 중 45개 데이터밖에 가져오지 못했습니다!') 
else : 
    print('모든 데이터를 가져왔습니다!') 
    print(places)

9415 개 중 45개 데이터밖에 가져오지 못했습니다!
